In [1]:
import pandas as pd
import re
import xmltodict
import flatdict
import os
import sys

In [2]:
class f4data:
    """
    Create a class for holding formatted Form-4 data
    
    """
       
    issuer_list = ["issuerCik", "issuerName", "issuerTradingSymbol"]
    
    reporting_list = [
        "reportingOwnerId.rptOwnerCik",
        "reportingOwnerId.rptOwnerName",
        "reportingOwnerAddress.rptOwnerStreet1",
        "reportingOwnerAddress.rptOwnerStreet2",
        "reportingOwnerAddress.rptOwnerCity",
        "reportingOwnerAddress.rptOwnerState",
        "reportingOwnerAddress.rptOwnerZipCode",
        "reportingOwnerAddress.rptOwnerStateDescription",
        "reportingOwnerRelationship.isDirector",
        "reportingOwnerRelationship.isOfficer",
        "reportingOwnerRelationship.isTenPercentOwner",
        "reportingOwnerRelationship.isOther",
        "reportingOwnerRelationship.officerTitle",
        "reportingOwnerRelationship.otherText"
        ]

    nonDerivative_list = [
        "securityTitle.value",
        "transactionDate.value",
        "deemedExecutionDate.value",
        "transactionCoding.transactionCode",
        "transactionTimeliness.value",
        "transactionAmounts.transactionShares.value",
        "transactionAmounts.transactionAcquiredDisposedCode.value",
        "transactionAmounts.transactionPricePerShare.value",
        "postTransactionAmounts.sharesOwnedFollowingTransaction.value",
        "ownershipNature.directOrIndirectOwnership.value",
        "ownershipNature.natureOfOwnership.value"
        ]
        
    derivative_list = [
        "securityTitle.value",
        "conversionOrExercisePrice.value",
        "transactionDate.value",
        "deemedExecutionDate.value",
        "transactionCoding.transactionCode",
        "transactionTimeliness.value",
        "transactionAmounts.transactionAcquiredDisposedCode.value",
        "transactionAmounts.transactionShares.value",
        "exerciseDate.value",
        "expirationDate.value",
        "underlyingSecurity.underlyingSecurityTitle.value",
        "underlyingSecurity.underlyingSecurityShares.value",
        "transactionAmounts.transactionPricePerShare.value",
        "postTransactionAmounts.sharesOwnedFollowingTransaction.value",
        "ownershipNature.directOrIndirectOwnership.value",
        "ownershipNature.natureOfOwnership.value"
        ]
   
    footnotes_list = ["footnote_"]
    
    
    def __init__(self, table_name, orig_df):
        """
        This function creates a DataFrame, with standardized column names, and dropped redundant entries
        
        table_name: string, name of database to create
        orig_df:    pandas DataFrame, full table contains all the data columns        
        """
        if table_name == "nonDerivative":
            column_list = self.issuer_list + self.reporting_list + self.nonDerivative_list
        elif table_name == "derivative":
            column_list = self.issuer_list + self.reporting_list + self.derivative_list
        elif table_name == "footnotes":
            column_list = self.issuer_list + self.reporting_list + self.footnotes_list
        else:
            raise ValueError("Unknown table name!")
            
        # could do some more checking with column_names after concat         
        empty_df = pd.DataFrame(columns=column_list)
        self.df  = pd.concat([empty_df,orig_df])[column_list]

        self.check_colname(empty_df, orig_df)


    def check_colname(self, empty_df, orig_df):
        """
        This function checks if the code is reading new unknown column names
        
        empty_df: pandas DataFrame, contains empty DataFrame with columns that have standard names
        orig_df:  pandas DataFrame, full table contains all the data columns 
        """

        # Exclude already known list of outliers:
        #         footnote
        #         transactionCoding.transactionFormType (4 for these forms, not included in database)
        #         transactionCoding.equitySwapInvolved (field not used by form 4)
        #         transactionTimeliness (if it's still here, it's a duplicate)
        #         deemedExecutionDate, empty field, populated field should be deemedExecutionDate.value
        #         transactionTimeliness, empty field, pupulated field should be transactionTimeliness.value

        out_list = list(set(orig_df.columns.values) - set(empty_df.columns.values))
        for coln in out_list:
            i = coln.lower()
            if ("footnote" not in i and "equityswap" not in i
            and "formtype" not in i and "transactiontimeliness" not in i
            and "transactiontimeliness" not in i and "deemedexecutiondate" not in i):
                print("Unmatched column name: "+coln)
        
        return


In [3]:
def proc_form4txt(filepath, filename):
    """
    This function processes the xml text for correct reading later using flatdict:
    1. Add a few lines to xml, so that flatdict can process things correctly
    2. Merge "Holding" to "Transaction", so that no separate form is needed
    3. Edit <footnotes> for flatdict to read
    
    filepath: string, directory for file
    filename: string, full filename of Form-4.txt for pre-processing
    """
    
    fileloc = filepath+filename
    infile  = open(fileloc, 'r')
    lines   = infile.readlines()
    
    outfile = open(fileloc+'.mod','w')
    for line in lines:
        # add line so that flatdic can process all as a list
        if r'</nonDerivativeTable>' in line and r'<nonDerivativeTable></nonDerivativeTable>' not in line:
            outfile.write(r'<nonDerivativeTransaction></nonDerivativeTransaction>' + "\n") 
        if r'</derivativeTable>' in line and r'<derivativeTable></derivativeTable>' not in line:
            outfile.write(r'<derivativeTransaction></derivativeTransaction>' + "\n") 
        if r'</footnotes>' in line and r'<footnotes></footnotes>' not in line:
            outfile.write(line.replace('</footnotes>', '<footnote><footnote_>  </footnote_></footnote></footnotes>'))
            continue
            
        # "Holding" and "Transaction" are slight variation of same table
        if 'nonDerivativeHolding' in line:
            outfile.write(line.replace('nonDerivativeHolding', 'nonDerivativeTransaction'))
            continue
        if 'derivativeHolding' in line:
            outfile.write(line.replace('derivativeHolding', 'derivativeTransaction'))
            continue
        
        # add additional nesting in footnote, so that flatdic process and separate the notes
        if r'<footnote ' in line:
            outfile.write(line.replace(' id', '><footnote_>id').replace('</footnote>', '</footnote_></footnote>'))
            continue
        if r'</footnote>' in line:
            outfile.write(line.replace('</footnote>', '</footnote_></footnote>'))
            continue
              
        outfile.write(line)
        
    outfile.close()
    infile.close()
    
    return


def form4xml_toflatdict(filepath, filename):
    
    with open(filepath+filename) as f:
        data = f.read()

    # extract file around ownershipDocument 
    matcher = re.compile('<\?xml.*ownershipDocument>', flags=re.MULTILINE|re.DOTALL)
    matches = matcher.search(data)
    xml     = matches.group(0)

    # load entire xml to dict object
    xmldict = xmltodict.parse(xml)

    # use flatdict tool to flatten levels of dictionary for easy indexing
    return flatdict.FlatDict(xmldict["ownershipDocument"], delimiter='.')


def flatdict_toDF(table_d):
    """
    This function takes a flat dictionary object and process it as follows: 
    If there is only 1 item, it is a dictionary. Convert it to a pandas DF object
    If there are more than 1 item, it will be a list. We flat it further and convert it to a pandas DF object
    
    table_d: could be list or a pandas DataFrame that has two rows (one of them contains column names)
    return:  pandas DataFrame with column names
    """
    
    if isinstance(table_d, list):
        d_list = []
        for i in table_d:
            d_list.append(flatdict.FlatDict(i, delimiter='.'))
        
        return pd.DataFrame(d_list)
    
    else:
        tmp_df   = pd.DataFrame(table_d.items()).T
        col_name = tmp_df.iloc[0] 
    
        return tmp_df.drop([0]).reset_index(drop=True).rename(col_name, axis=1)


def form4df_tocsv(filepath, full_dict, issuer_df, reportingOwner_df):
    """
    This function takes read-in information and save it to .csv database
    
    filepath:          string, directory for file
    full_dict:         flatdict, contains full flatdic read from xml 
    issuer_df:         pandas DataFrame, contains issuer info
    reportingOwner_df: pandas DataFrame, contains reporting owner info
    """

    exist_nonDer = False
    exist_der = False

    # work on nonDerivativeTable
    if "nonDerivativeTable.nonDerivativeTransaction" in full_dict.keys():
        nonDerivativeTable_df = flatdict_toDF(full_dict["nonDerivativeTable.nonDerivativeTransaction"])
        # add information about issuer and owner to the tables
        nonDerivative_cDF = concat_abtoC(issuer_df, reportingOwner_df, nonDerivativeTable_df)
        # remove the last row that was added for flatdict reading
        f4_nonDerivative  = f4data("nonDerivative", nonDerivative_cDF.iloc[:-1])
        save_dftocsv(filepath, "nonDerivative.csv", f4_nonDerivative.df)
        exist_nonDer = True
        
    # work on derivativeTable
    if "derivativeTable.derivativeTransaction" in full_dict.keys():
        derivativeTable_df= flatdict_toDF(full_dict["derivativeTable.derivativeTransaction"])
        derivative_cDF    = concat_abtoC(issuer_df, reportingOwner_df, derivativeTable_df)
        f4_derivative     = f4data("derivative", derivative_cDF.iloc[:-1])
        save_dftocsv(filepath, "derivative.csv", f4_derivative.df)
        exist_der = True
    
    # work on footnotes
    if "footnotes.footnote" in full_dict.keys():
        footnotes_df  = flatdict_toDF(full_dict["footnotes.footnote"])
        footnotes_cDF = concat_abtoC(issuer_df, reportingOwner_df, footnotes_df)
        f4_footnotes  = f4data("footnotes", footnotes_cDF.iloc[:-1])
        # add transaction date to footnotes table
        row = len(f4_footnotes.df.index)
        if exist_nonDer:
            transactionDate = [f4_nonDerivative.df["transactionDate.value"].iloc[0]]*row
        elif exist_der:
            transactionDate = [f4_derivative.df["transactionDate.value"].iloc[0]]*row
        else:
            raise Exception("Try to find transaction date for footnotes. Empty entries for nonDerivative and derivative tables.")
        footnotes_withdate = f4_footnotes.df
        footnotes_withdate["transactionDate"] = transactionDate
        save_dftocsv(filepath, "footnotes.csv", footnotes_withdate)
    
    return


def concat_abtoC(a, b, c):
    """
    This function takes three pandas DF objects a b and c (a and b has one row and c may have multiple rows) to:
    1. Duplicate lines of a and b to the same number of rows in c
    2. Merge a b and c to a large DF along axis=1
    
    a:      pandas DataFrame, one row only
    b:      pandas DataFrame, one row only
    c:      pandas DataFrame, may have multiple rows
    return: pandas DataFrame
    """
    
    n   = len(c.index)
    t1  = pd.concat([a]*n,ignore_index=True)
    t2  = pd.concat([b]*n,ignore_index=True)
    t12 = pd.concat([t1.reset_index(drop=True),t2.reset_index(drop=True)], axis=1)
    
    return pd.concat([t12.reset_index(drop=True),c.reset_index(drop=True)], axis=1)


def save_dftocsv(filepath, filename, df):
    """
    This function takes a pandas DF and save to specific filename in filepath.
    Append if file already exists.
    
    filepath: string, directory for file
    filename: string, full filename for csv output   
    """
    
    fileloc = filepath+filename
    file_present = os.path.isfile(fileloc) 
    if file_present:
        df.to_csv(fileloc, index=False, mode='a', header=False)    
    else:
        df.to_csv(fileloc, index=False)
        
    return


In [4]:
def form4_tocsv(inpath, outpath, filename):
    """
    This is the main function that reads form 4 file and process it and save it to .csv database
    
    inpath:   string, directory for input files
    outpath:  string, directory for output files
    filename: string, full filename of Form-4.txt after pre-processing
    """
    
    # pre-processing .txt file, so that xml can be formatted properly with flatdict
    proc_form4txt(inpath, filename)
    
    # extract xml information to flatdict object
    full_dict = form4xml_toflatdict(inpath, filename+'.mod')

    # create subsections from the full dictionary
    # issuer and reportingOwner first; hopefully these fields are populated
    issuer_df = flatdict_toDF(full_dict["issuer"])
          
    if isinstance(full_dict["reportingOwner"], list):
        for item in full_dict["reportingOwner"]:
            tmp = flatdict.FlatDict(item, delimiter='.')
            reportingOwner_df = flatdict_toDF(tmp)
            form4df_tocsv(outpath, full_dict, issuer_df, reportingOwner_df)
        
        # # DEBUG only: use only one reporting Owner for multiple owner cases
        # item=full_dict["reportingOwner"][0]
        # tmp = flatdict.FlatDict(item, delimiter='.')
        # reportingOwner_df = flatdict_toDF(tmp)
        # form4df_tocsv(outpath, full_dict, issuer_df, reportingOwner_df)
        # # DEBUG only

    else:
        reportingOwner_df = flatdict_toDF(full_dict["reportingOwner"])
        form4df_tocsv(outpath, full_dict, issuer_df, reportingOwner_df)
        
    return



In [5]:
# read_form4txt("tmp.txt")

filepath="./test-jup/test_data/"
filename ="912728_4_0000912728-20-000168.txt"
inpath=filepath
outpath=filepath
# form4_tocsv(inpath, outpath, filename)


In [6]:
filepath="./test-jup/test_data/"
inpath=filepath
outpath=filepath

infile  = open(inpath+"list_txt", 'r')
lines   = infile.readlines()
for line in lines:
    filename = line.strip()
    print(filename)
    form4_tocsv(inpath, outpath, filename)
infile.close()


1023844_1_0001437749-20-000181.txt
1034604_2_0001209191-20-036702.txt
104169_4_0001127602-20-030022.txt
1047122_1_0001047122-20-000051.txt
1049521_1_0001562180-20-002168.txt
1058725_3_0001712184-20-000269.txt
1084991_2_0001084991-20-000022.txt
1109354_3_0001179110-20-009902.txt
1136554_4_0001209191-20-061181.txt
1158369_2_0001567619-20-011536.txt
1163370_2_0001761053-20-000002.txt
1192933_2_0001179110-20-005642.txt
1221787_2_0001209191-20-035900.txt
1222848_4_0000746598-20-000179.txt
1227500_1_0000899243-20-004098.txt
1278895_1_0000899243-20-009380.txt
1286331_1_0001286331-20-000001.txt
1333986_1_0001209191-20-017244.txt
1336745_4_0001213900-20-040637.txt
1363364_2_0001638599-20-000500.txt
1370450_1_0001179110-20-000668.txt
1395942_1_0001395942-20-000033.txt
1403161_2_0001127602-20-018860.txt
1403679_2_0001179110-20-006906.txt
1412408_1_0000899243-20-001780.txt
1430231_1_0000899243-20-001255.txt
1447887_1_0001393726-20-000022.txt
1448238_1_0001357615-20-000104.txt
1467858_2_0001467858-

In [10]:
nd = pd.read_csv("./test-jup/test_data/nonDerivative.csv")
d = pd.read_csv("./test-jup/test_data/derivative.csv")
f = pd.read_csv("./test-jup/test_data/footnotes.csv")
display(f)


,issuerCik,issuerName,issuerTradingSymbol,reportingOwnerId.rptOwnerCik,reportingOwnerId.rptOwnerName,reportingOwnerAddress.rptOwnerStreet1,reportingOwnerAddress.rptOwnerStreet2,reportingOwnerAddress.rptOwnerCity,reportingOwnerAddress.rptOwnerState,reportingOwnerAddress.rptOwnerZipCode,reportingOwnerAddress.rptOwnerStateDescription,reportingOwnerRelationship.isDirector,reportingOwnerRelationship.isOfficer,reportingOwnerRelationship.isTenPercentOwner,reportingOwnerRelationship.isOther,reportingOwnerRelationship.officerTitle,reportingOwnerRelationship.otherText,footnote_,transactionDate
0,1348036,"AVALARA, INC.",AVLR,1034604,SHARPLES BRIAN,255 SOUTH KING ST.,SUITE 1800,SEATTLE,WA,98104,NaN,1,0,0,0,NaN,NaN,"id=""F1"">The option vests and becomes exercisab...",2020-06-11
1,104169,Walmart Inc.,WMT,1013440,WALTON ALICE L,P.O. BOX 1860,NaN,BENTONVILLE,AR,72712,NaN,NaN,NaN,1,NaN,NaN,NaN,"id=""F1"">This sale from the Walton Family Holdi...",2020-11-23
2,104169,Walmart Inc.,WMT,1013440,WALTON ALICE L,P.O. BOX 1860,NaN,BENTONVILLE,AR,72712,NaN,NaN,NaN,1,NaN,NaN,NaN,"id=""F2"">This sale from the Trust was executed ...",2020-11-23
3,104169,Walmart Inc.,WMT,1013440,WALTON ALICE L,P.O. BOX 1860,NaN,BENTONVILLE,AR,72712,NaN,NaN,NaN,1,NaN,NaN,NaN,"id=""F3"">On November 24, 2020, the reporting pe...",2020-11-23
4,104169,Walmart Inc.,WMT,1013440,WALTON ALICE L,P.O. BOX 1860,NaN,BENTONVILLE,AR,72712,NaN,NaN,NaN,1,NaN,NaN,NaN,"id=""F4"">This sale from the Trust was executed ...",2020-11-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,793733,SKYWEST INC,SKYW,1546254,Siegfried Meredith R.,444 SOUTH RIVER ROAD,NaN,ST GEORGE,UT,84790,NaN,1,0,0,0,NaN,NaN,"id=""F1"">Shares issued pursuant to the SkyWest,...",2020-02-04
374,897802,"SPECIAL OPPORTUNITIES FUND, INC.",SPE,1067621,GOLDSTEIN PHILLIP,60 HERITAGE DRIVE,NaN,PLEASANTVILLE,NY,10570,NaN,1,1,0,0,Chairman and Secretary,NaN,"id=""F1"">The Reporting Person disclaims benefic...",2020-07-15
375,899866,"ALEXION PHARMACEUTICALS, INC.",ALXN,1792124,Carino Tanisha,"C/O ALEXION PHARMACEUTICALS, INC.",121 SEAPORT BOULEVARD,BOSTON,MA,02210,NaN,0,1,0,0,EVP & CCAO,NaN,"id=""F1"">Award of Restricted Stock Units under ...",2020-02-28
376,912728,FORWARD AIR CORP,FWRD,1182297,RUBLE CHRIS C,1915 SNAPPS FERRY ROAD,BUILDING N,GREENEVILLE,TN,37745,NaN,0,1,0,0,Chief Operating Officer,NaN,"id=""F1"">This option vests 33-1/3% each year ov...",2020-11-27
